# Agentic Appraisal System

## Overview

This notebook demonstrates an AI-powered automobile repair cost estimation system. The system processes First Notice of Loss (FNOL) data to automatically generate detailed repair cost estimates for collision-damaged vehicles.

- Extracts vehicle information from FNOL JSON documents
- Classifies vehicles into standard categories (Car, Truck, SUV, Van)
- Calculates repair costs based on industry-standard pricing tiers
- Applies cost multipliers for luxury brands and vehicle age
- Generates comprehensive JSON repair estimates

## Workflow

1. **FNOL Data Loading** - Load and parse First Notice of Loss JSON file
2. **Vehicle Classification** - Determine vehicle type for appropriate pricing tier
3. **Damage Assessment** - Identify all damaged components from FNOL data
4. **Cost Calculation** - Apply pricing matrix and multipliers to estimate repair costs
5. **Report Generation** - Create structured JSON estimate with detailed breakdown
6. **Agent Execution** - Strands agent autonomously invokes estimation tool and provides summary



In [1]:
# Standard library imports
import json
from datetime import datetime

# AWS SDK
import boto3

# Strands Agents SDK
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool

print("✓ All dependencies imported successfully")

✓ All dependencies imported successfully


In [2]:
# AWS Configuration
AWS_REGION = "us-east-1"
MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"

# Initialize Bedrock client to verify connectivity
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)

print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
print(f"✓ Using model: {MODEL_ID}")

✓ AWS Bedrock configured for region: us-east-1
✓ Using model: anthropic.claude-sonnet-4-5-20250929-v1:0


In [3]:
# Load FNOL JSON data
with open('data/FNOL.json', 'r') as f:
    fnol_data = json.load(f)

# Extract vehicle information
insured_vehicle = fnol_data['vehicle']['insuredVehicle']
vehicle_year = insured_vehicle['year']
vehicle_make = insured_vehicle['make']
vehicle_model = insured_vehicle['model']
vehicle_vin = insured_vehicle['vin']
vehicle_mileage = insured_vehicle['mileage']

# Extract damage information
damage_info = insured_vehicle['damage']
damage_areas = damage_info['areas']
damage_description = damage_info['description']

# Extract claim number for report metadata
claim_number = fnol_data['fnol']['claimNumber']

# Display extracted information

print("FNOL DATA LOADED SUCCESSFULLY")


FNOL DATA LOADED SUCCESSFULLY


## Vehicle Type Classification

Classify the vehicle into one of four standard categories for pricing purposes:
- **Car**: Sedans, coupes, hatchbacks (default category)
- **Truck**: Pickup trucks (F-150, Silverado, Ram, etc.)
- **SUV**: Sport utility vehicles (Explorer, Pilot, CR-V, RAV4, etc.)
- **Van**: Minivans and cargo vans (Odyssey, Sienna, Transit, etc.)

The classification determines the base pricing tier for repair cost estimation.

In [4]:
def classify_vehicle_type(make: str, model: str) -> str:
    """
    Classify vehicle into type category for pricing purposes.
    
    Args:
        make: Vehicle manufacturer (e.g., "Honda", "Ford")
        model: Vehicle model name (e.g., "Accord", "F-150")
        
    Returns:
        One of: "Car", "Truck", "SUV", "Van"
    """
    # Normalize inputs for case-insensitive matching
    model_lower = model.lower()
    
    # SUV classification - common SUV models
    suv_keywords = [
        'explorer', 'pilot', 'cr-v', 'crv', 'rav4', 'highlander',
        'tahoe', 'suburban', 'expedition', '4runner', 'pathfinder',
        'traverse', 'durango', 'grand cherokee', 'cherokee', 'wrangler',
        'rogue', 'murano', 'armada', 'sequoia', 'land cruiser',
        'cx-5', 'cx-9', 'outback', 'forester', 'ascent',
        'santa fe', 'tucson', 'palisade', 'telluride', 'sorento',
        'sportage', 'atlas', 'tiguan', 'touareg', 'x5', 'x3',
        'q5', 'q7', 'gx', 'rx', 'nx', 'xt5', 'xt6', 'escalade'
    ]
    
    # Truck classification - pickup trucks
    truck_keywords = [
        'f-150', 'f150', 'f-250', 'f250', 'f-350', 'f350',
        'silverado', 'sierra', 'ram', 'tundra', 'tacoma',
        'ranger', 'colorado', 'canyon', 'frontier', 'titan',
        'ridgeline', 'gladiator', 'maverick'
    ]
    
    # Van classification - minivans and cargo vans
    van_keywords = [
        'odyssey', 'sienna', 'pacifica', 'caravan', 'grand caravan',
        'transit', 'sprinter', 'promaster', 'metris', 'express',
        'savana', 'nv', 'quest', 'sedona', 'carnival'
    ]
    
    # Check for SUV
    for keyword in suv_keywords:
        if keyword in model_lower:
            return "SUV"
    
    # Check for Truck
    for keyword in truck_keywords:
        if keyword in model_lower:
            return "Truck"
    
    # Check for Van
    for keyword in van_keywords:
        if keyword in model_lower:
            return "Van"
    
    # Default to Car for sedans, coupes, hatchbacks, and unknown models
    return "Car"


# Classify the vehicle from FNOL data
vehicle_type = classify_vehicle_type(vehicle_make, vehicle_model)

print("=" * 60)
print("VEHICLE TYPE CLASSIFICATION")
print("=" * 60)
print(f"\n🚗 Vehicle: {vehicle_year} {vehicle_make} {vehicle_model}")
print(f"📊 Classification: {vehicle_type}")
print(f"\n💡 This classification determines the base pricing tier for repair estimates.")
print("\n" + "=" * 60)

VEHICLE TYPE CLASSIFICATION

🚗 Vehicle: 2020 Honda Accord
📊 Classification: Car

💡 This classification determines the base pricing tier for repair estimates.



## Repair Cost Estimation Tool

Define the core estimation tool using the Strands `@tool` decorator. This tool will:
- Accept vehicle and damage information as parameters
- Calculate repair costs based on vehicle type and damaged components
- Apply cost multipliers for luxury brands and vehicle age
- Return a structured JSON estimate with detailed breakdown

The agent will autonomously invoke this tool when asked to estimate repair costs.

In [5]:
@tool
def estimate_repair_costs(
    vehicle_year: int,
    vehicle_make: str,
    vehicle_model: str,
    vehicle_vin: str,
    vehicle_mileage: int,
    damage_areas: list,
    damage_description: str,
    claim_number: str
) -> str:
    """
    Estimate repair costs for a collision-damaged vehicle based on vehicle type,
    damaged components, and applicable cost multipliers.
    
    This tool analyzes vehicle information and damage details to generate a comprehensive
    repair cost estimate. It classifies the vehicle into a pricing tier (Car, Truck, SUV, Van),
    calculates costs for each damaged component, applies multipliers for luxury brands and
    vehicle age, and returns a detailed JSON estimate.
    
    Args:
        vehicle_year: Year of vehicle manufacture (e.g., 2020)
        vehicle_make: Vehicle manufacturer name (e.g., "Honda", "Ford", "BMW")
        vehicle_model: Vehicle model name (e.g., "Accord", "F-150", "X5")
        vehicle_vin: Vehicle Identification Number (17-character alphanumeric)
        vehicle_mileage: Current vehicle mileage in miles (e.g., 35650)
        damage_areas: List of damaged components (e.g., ["Rear bumper", "Trunk", "Taillights"])
        damage_description: Detailed description of damage (e.g., "Rear bumper dented and paint scratched")
        claim_number: Insurance claim number for report metadata (e.g., "CL-2023-1156789")
    
    Returns:
        JSON string containing:
        - estimate_metadata: Claim number, estimate date, disclaimer
        - vehicle_information: Year, make, model, VIN, mileage, vehicle type classification
        - damage_assessment: Array of damaged components with individual costs and subtotal
        - cost_adjustments: Applied multipliers (luxury brand, age discount) with reasons
        - total_estimate: Final estimated repair cost in USD
    
    Example:
        >>> estimate = estimate_repair_costs(
        ...     vehicle_year=2020,
        ...     vehicle_make="Honda",
        ...     vehicle_model="Accord",
        ...     vehicle_vin="1HGCV2F35LA007149",
        ...     vehicle_mileage=35650,
        ...     damage_areas=["Rear bumper", "Trunk", "Taillights"],
        ...     damage_description="Rear-end collision damage",
        ...     claim_number="CL-2023-1156789"
        ... )
    """
    try:
        # Validate required parameters
        missing_fields = []
        if not vehicle_year:
            missing_fields.append("vehicle_year")
        if not vehicle_make:
            missing_fields.append("vehicle_make")
        if not vehicle_model:
            missing_fields.append("vehicle_model")
        if not vehicle_vin:
            missing_fields.append("vehicle_vin")
        if not damage_areas or len(damage_areas) == 0:
            missing_fields.append("damage_areas")
        if not claim_number:
            missing_fields.append("claim_number")
        
        if missing_fields:
            return json.dumps({
                "error": "Missing required fields",
                "missing_fields": missing_fields,
                "status": "validation_failed"
            })
        
        # Classify vehicle type for pricing
        vehicle_type = classify_vehicle_type(vehicle_make, vehicle_model)
        
        # Calculate cost multipliers
        multipliers = []
        cumulative_multiplier = 1.0
        
        # Luxury brand multiplier (1.3x)
        luxury_brands = [
            "lexus", "bmw", "mercedes-benz", "mercedes", "audi", "porsche",
            "jaguar", "land rover", "cadillac", "lincoln", "acura",
            "infiniti", "genesis"
        ]
        
        if vehicle_make.lower() in luxury_brands:
            luxury_multiplier = 1.3
            cumulative_multiplier *= luxury_multiplier
            multipliers.append({
                "type": "luxury_brand",
                "factor": luxury_multiplier,
                "reason": f"{vehicle_make} is a luxury brand with higher parts and labor costs"
            })
        
        # Age discount multiplier (0.85x for vehicles older than 10 years)
        current_year = datetime.now().year
        vehicle_age = current_year - vehicle_year
        
        if vehicle_age > 10:
            age_multiplier = 0.85
            cumulative_multiplier *= age_multiplier
            multipliers.append({
                "type": "age_discount",
                "factor": age_multiplier,
                "reason": f"Vehicle is {vehicle_age} years old (older than 10 years), parts costs are lower"
            })
        
        # Calculate costs for each damaged component
        damaged_components = []
        subtotal = 0.0
        
        for component in damage_areas:
            base_cost = get_component_cost(component, vehicle_type)
            
            if base_cost == 0.0:
                # Component not found in pricing matrix, skip it
                continue
            
            # Apply multipliers to component cost
            adjusted_cost = base_cost * cumulative_multiplier
            subtotal += adjusted_cost
            
            damaged_components.append({
                "component": component,
                "damage_description": damage_description,
                "estimated_cost": round(adjusted_cost, 2)
            })
        
        # Calculate adjustment total (difference from base subtotal)
        base_subtotal = subtotal / cumulative_multiplier if cumulative_multiplier != 1.0 else subtotal
        adjustment_total = subtotal - base_subtotal
        
        # Generate JSON output
        estimate = {
            "estimate_metadata": {
                "claim_number": claim_number,
                "estimate_date": datetime.now().isoformat(),
                "disclaimer": "This is a preliminary estimate. Actual repair costs may vary based on hidden damage, parts availability, and labor rates."
            },
            "vehicle_information": {
                "year": vehicle_year,
                "make": vehicle_make,
                "model": vehicle_model,
                "vin": vehicle_vin,
                "mileage": vehicle_mileage,
                "vehicle_type": vehicle_type
            },
            "damage_assessment": {
                "damaged_components": damaged_components,
                "subtotal": round(subtotal, 2)
            },
            "cost_adjustments": {
                "multipliers": multipliers,
                "adjustment_total": round(adjustment_total, 2)
            },
            "total_estimate": {
                "amount": round(subtotal, 2),
                "currency": "USD"
            }
        }
        
        return json.dumps(estimate, indent=2)
        
    except Exception as e:
        return json.dumps({
            "error": "Cost calculation failed",
            "details": str(e),
            "status": "calculation_failed"
        })



## Pricing Matrix for Damage Components

Define the pricing matrix that maps damaged components to cost ranges based on vehicle type.
Each component has a minimum and maximum cost for each vehicle category (Car, Truck, SUV, Van).

### Pricing Tiers

- **Car**: Base pricing tier for sedans, coupes, and hatchbacks
- **Truck**: Higher pricing for pickup trucks due to larger parts and labor
- **SUV**: Mid-to-high pricing for sport utility vehicles
- **Van**: Mid-range pricing for minivans and cargo vans

### Cost Calculation

For each damaged component, the tool uses the **midpoint** of the cost range for the vehicle type.
This provides a balanced estimate that accounts for typical repair scenarios.

In [6]:
# Pricing matrix: Component costs by vehicle type
# Format: {component: {vehicle_type: (min_cost, max_cost)}}
PRICING_MATRIX = {
    # Bumpers - Front and rear impact protection
    "front bumper": {
        "Car": (800, 1500),
        "Truck": (1200, 2000),
        "SUV": (1000, 1800),
        "Van": (900, 1600)
    },
    "rear bumper": {
        "Car": (800, 1500),
        "Truck": (1200, 2000),
        "SUV": (1000, 1800),
        "Van": (900, 1600)
    },
    
    # Hood - Engine compartment cover
    "hood": {
        "Car": (1000, 2000),
        "Truck": (1500, 3000),
        "SUV": (1200, 2500),
        "Van": (1100, 2200)
    },
    
    # Trunk - Rear storage compartment
    "trunk": {
        "Car": (1200, 2500),
        "Truck": (1800, 3500),
        "SUV": (1500, 3000),
        "Van": (1400, 2800)
    },
    
    # Doors - Per door pricing
    "door": {
        "Car": (1500, 3000),
        "Truck": (2000, 4000),
        "SUV": (1800, 3500),
        "Van": (1700, 3200)
    },
    
    # Side panels - Per panel pricing
    "side panel": {
        "Car": (1800, 3500),
        "Truck": (2500, 5000),
        "SUV": (2200, 4500),
        "Van": (2000, 4000)
    },
    
    # Fenders - Per fender pricing
    "fender": {
        "Car": (900, 2000),
        "Truck": (1400, 2800),
        "SUV": (1200, 2500),
        "Van": (1000, 2200)
    },
    
    # Headlights - Per light pricing
    "headlight": {
        "Car": (400, 800),
        "Truck": (600, 1200),
        "SUV": (500, 1000),
        "Van": (450, 900)
    },
    
    # Taillights - Per light pricing
    "taillight": {
        "Car": (300, 600),
        "Truck": (500, 800),
        "SUV": (400, 700),
        "Van": (350, 650)
    },
    
    # Grille - Front air intake
    "grille": {
        "Car": (400, 900),
        "Truck": (700, 1500),
        "SUV": (600, 1200),
        "Van": (500, 1000)
    },
    
    # Radiator - Cooling system
    "radiator": {
        "Car": (800, 1500),
        "Truck": (1200, 2500),
        "SUV": (1000, 2000),
        "Van": (900, 1800)
    },
    
    # Frame damage - Structural damage (most expensive)
    "frame": {
        "Car": (2000, 5000),
        "Truck": (3000, 8000),
        "SUV": (2500, 6500),
        "Van": (2200, 6000)
    }
}


def get_component_cost(component: str, vehicle_type: str) -> float:
    """
    Get the estimated cost for a damaged component based on vehicle type.
    Uses the midpoint of the cost range for the estimate.
    
    Args:
        component: Name of damaged component (e.g., "rear bumper", "hood")
        vehicle_type: Vehicle classification ("Car", "Truck", "SUV", "Van")
    
    Returns:
        Estimated cost as float (midpoint of range)
    """
    # Normalize component name to lowercase for matching
    component_lower = component.lower().strip()
    
    # Handle common variations in component naming
    component_mapping = {
        "front bumper": "front bumper",
        "rear bumper": "rear bumper",
        "bumper": "rear bumper",  # Default to rear if not specified
        "hood": "hood",
        "trunk": "trunk",
        "door": "door",
        "doors": "door",
        "side panel": "side panel",
        "panel": "side panel",
        "fender": "fender",
        "fenders": "fender",
        "headlight": "headlight",
        "headlights": "headlight",
        "taillight": "taillight",
        "taillights": "taillight",
        "tail light": "taillight",
        "tail lights": "taillight",
        "grille": "grille",
        "grill": "grille",
        "radiator": "radiator",
        "frame": "frame",
        "frame damage": "frame"
    }
    
    # Map component to standard name
    standard_component = component_mapping.get(component_lower, component_lower)
    
    # Look up pricing
    if standard_component in PRICING_MATRIX:
        min_cost, max_cost = PRICING_MATRIX[standard_component][vehicle_type]
        # Return midpoint of range
        return (min_cost + max_cost) / 2
    
    # If component not found, return 0 (will be handled in error reporting)
    return 0.0


sample_components = ["rear bumper", "hood", "door", "taillight", "frame"]
for component in sample_components:
    car_cost = get_component_cost(component, "Car")
    truck_cost = get_component_cost(component, "Truck")
    suv_cost = get_component_cost(component, "SUV")
    van_cost = get_component_cost(component, "Van")
    
    print(f"{component.title():<20} ${car_cost:>6,.0f}        ${truck_cost:>6,.0f}        ${suv_cost:>6,.0f}        ${van_cost:>6,.0f}")



Rear Bumper          $ 1,150        $ 1,600        $ 1,400        $ 1,250
Hood                 $ 1,500        $ 2,250        $ 1,850        $ 1,650
Door                 $ 2,250        $ 3,000        $ 2,650        $ 2,450
Taillight            $   450        $   650        $   550        $   500
Frame                $ 3,500        $ 5,500        $ 4,500        $ 4,100


In [7]:
# Initialize Bedrock model
model = BedrockModel(
    # model_id="anthropic.claude-sonnet-4-5-20250929-v1:0",
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="us-east-1"
)


In [20]:
# Create agent with repair cost estimation tool
agent = Agent(
    name="Repair Cost Estimator",
    model=model,
    tools=[estimate_repair_costs],
    system_prompt="""You are an expert automotive claims adjuster specializing in repair cost estimation.
    
When provided with vehicle damage information, you should:
1. Use the estimate_repair_costs tool to generate detailed cost estimates and return JSON estimate
2. Analyze the returned JSON estimate carefully
3. Provide a clear, professional summary of the repair costs
4. Highlight key factors affecting the estimate (vehicle type, luxury brand, age, etc.)
5. Explain the breakdown of costs by damaged component
6. Note any cost adjustments and their reasons
7. Present the total estimated repair cost prominently

Your responses should be:
- Professional and authoritative
- Clear and easy to understand for insurance adjusters
- Detailed enough to justify the estimate
- Transparent about the preliminary nature of the estimate
- Include a section to provide the full JSON estimate

Always remind users that this is a preliminary estimate and actual costs may vary based on hidden damage, parts availability, and labor rates."""
)



In [21]:
# Create user prompt with vehicle and damage information
user_prompt = f"""
Please estimate the repair costs for the following vehicle damage claim:

**Claim Information:**
- Claim Number: {claim_number}
- Incident Date: {fnol_data['incident']['date']}
- Incident Description: {fnol_data['incident']['description']}

**Vehicle Information:**
- Year: {vehicle_year}
- Make: {vehicle_make}
- Model: {vehicle_model}
- VIN: {vehicle_vin}
- Mileage: {vehicle_mileage:,} miles

**Damage Assessment:**
- Damaged Areas: {', '.join(damage_areas)}
- Damage Description: {damage_description}
- Vehicle Drivable: {insured_vehicle['drivable']}

Please provide a detailed repair cost estimate including:
1. Cost breakdown by damaged component
2. Any applicable cost adjustments (luxury brand, vehicle age, etc.)
3. Total estimated repair cost
4. Professional summary and recommendations
"""

print("=" * 80)
print("USER PROMPT CREATED")
print("=" * 80)
print(user_prompt)
print("=" * 80)
print("\n📝 Prompt ready for agent execution")
print("\n" + "=" * 80)

USER PROMPT CREATED

Please estimate the repair costs for the following vehicle damage claim:

**Claim Information:**
- Claim Number: CL-2023-1156789
- Incident Date: 2023-11-19
- Incident Description: While stopped at a red light, my vehicle was rear-ended by another car. The impact pushed my vehicle forward about 5 feet. I pulled over to the side of the road to exchange information with the other driver.

**Vehicle Information:**
- Year: 2020
- Make: Honda
- Model: Accord
- VIN: 1HGCV2F35LA007149
- Mileage: 35,650 miles

**Damage Assessment:**
- Damaged Areas: Rear bumper, Trunk, Taillights, Possible frame
- Damage Description: Rear bumper dented and paint scratched, trunk lid misaligned, possible frame damage, brake lights damaged
- Vehicle Drivable: False

Please provide a detailed repair cost estimate including:
1. Cost breakdown by damaged component
2. Any applicable cost adjustments (luxury brand, vehicle age, etc.)
3. Total estimated repair cost
4. Professional summary and reco

In [25]:
!curl http://localhost:9000/.well-known/agent-card.json 

{"capabilities":{"streaming":true},"defaultInputModes":["text"],"defaultOutputModes":["text"],"description":"A Single agent wit Appraisal tools capabilities","name":"Repair Cost Estimator","preferredTransport":"JSONRPC","protocolVersion":"0.3.0","skills":[{"description":"Estimate repair costs for a collision-damaged vehicle based on vehicle type,\ndamaged components, and applicable cost multipliers.\n\nThis tool analyzes vehicle information and damage details to generate a comprehensive\nrepair cost estimate. It classifies the vehicle into a pricing tier (Car, Truck, SUV, Van),\ncalculates costs for each damaged component, applies multipliers for luxury brands and\nvehicle age, and returns a detailed JSON estimate.\n\nArgs:\n    vehicle_year: Year of vehicle manufacture (e.g., 2020)\n    vehicle_make: Vehicle manufacturer name (e.g., \"Honda\", \"Ford\", \"BMW\")\n    vehicle_model: Vehicle model name (e.g., \"Accord\", \"F-150\", \"X5\")\n    vehicle_vin: Vehicle Identification Number

In [28]:
!curl -X POST http://0.0.0.0:9000 \-H "Content-Type: application/json" \-d '{"jsonrpc": "2.0",  "id": "req-001",  "method": "message/send",  "params": {  "message": {  "role": "user",  "parts": [  {  "kind": "text",  "text": user_prompt }  ],  "messageId": "d0673ab9-796d-4270-9435-451912020cd1"  }  } }' 

{"error":{"code":-32700,"message":"Expecting value: line 1 column 150 (char 149)"},"jsonrpc":"2.0"}

In [22]:
# Run the agent
response = agent(user_prompt)

print("\n" + "=" * 80)
print("✓ Agent execution completed successfully")
print("=" * 80)

I'll analyze this rear-end collision damage and provide you with a comprehensive repair cost estimate. Let me process the vehicle and damage information through our estimation system.
Tool #1: estimate_repair_costs
## Professional Repair Cost Estimate Summary

**Claim Number:** CL-2023-1156789  
**Estimate Date:** October 22, 2025  
**Vehicle:** 2020 Honda Accord (VIN: 1HGCV2F35LA007149)

---

### **TOTAL ESTIMATED REPAIR COST: $3,450.00 USD**

---

## Detailed Cost Breakdown by Component

| **Damaged Component** | **Estimated Cost** | **Description** |
|----------------------|-------------------|-----------------|
| **Rear Bumper** | $1,150.00 | Replacement and refinishing of rear bumper cover, including labor for removal, installation, and paint matching |
| **Trunk** | $1,850.00 | Trunk lid replacement or major repair due to misalignment, including adjustment of latch mechanisms and weatherstripping |  
| **Taillights** | $450.00 | Replacement of damaged brake light assemblies and a

In [23]:
# Display agent's response
print("=" * 80)
print("APPRAISAL AGENT - SUMMARY")
print("=" * 80)
print()
print(response)
print()
print("=" * 80)

APPRAISAL AGENT - SUMMARY

## Professional Repair Cost Estimate Summary

**Claim Number:** CL-2023-1156789  
**Estimate Date:** October 22, 2025  
**Vehicle:** 2020 Honda Accord (VIN: 1HGCV2F35LA007149)

---

### **TOTAL ESTIMATED REPAIR COST: $3,450.00 USD**

---

## Detailed Cost Breakdown by Component

| **Damaged Component** | **Estimated Cost** | **Description** |
|----------------------|-------------------|-----------------|
| **Rear Bumper** | $1,150.00 | Replacement and refinishing of rear bumper cover, including labor for removal, installation, and paint matching |
| **Trunk** | $1,850.00 | Trunk lid replacement or major repair due to misalignment, including adjustment of latch mechanisms and weatherstripping |  
| **Taillights** | $450.00 | Replacement of damaged brake light assemblies and associated wiring connections |

**Subtotal:** $3,450.00

## Cost Adjustments Analysis

**No adjustments applied** - This 2020 Honda Accord does not qualify for luxury brand multipliers, an

### Understanding the Estimate

The JSON estimate contains five main sections:

1. **Estimate Metadata**: Claim number, estimate date, and disclaimer about preliminary nature
2. **Vehicle Information**: Complete vehicle details including the classified vehicle type
3. **Damage Assessment**: Itemized list of damaged components with individual costs and subtotal
4. **Cost Adjustments**: Applied multipliers (luxury brand, age discount) with explanations
5. **Total Estimate**: Final repair cost in USD

### Cost Calculation Methodology

- **Base Costs**: Each component has a cost range based on vehicle type (Car, Truck, SUV, Van)
- **Midpoint Pricing**: The tool uses the midpoint of each range for balanced estimates
- **Luxury Multiplier**: 1.3x applied to luxury brands (Lexus, BMW, Mercedes, Audi, etc.)
- **Age Discount**: 0.85x applied to vehicles older than 10 years
- **Cumulative Effect**: Multipliers are applied cumulatively to all component costs

In [15]:
# Extract estimate in JSON
json_estimate = None

# Try to extract from agent's last run
# if hasattr(agent, '_last_run') and agent._last_run:
#     # Look for tool invocations in the run
#     for event in agent._last_run.get('events', []):
#         if event.get('type') == 'tool_result' and event.get('tool_name') == 'estimate_repair_costs':
#             json_estimate = event.get('result')
#             break

# If not found in agent history, call the tool directly for display purposes
if not json_estimate:
    print("📝 Calling estimation tool directly to retrieve JSON estimate...\n")
    json_estimate = estimate_repair_costs(
        vehicle_year=vehicle_year,
        vehicle_make=vehicle_make,
        vehicle_model=vehicle_model,
        vehicle_vin=vehicle_vin,
        vehicle_mileage=vehicle_mileage,
        damage_areas=damage_areas,
        damage_description=damage_description,
        claim_number=claim_number
    )

# Parse the JSON string
estimate_data = json.loads(json_estimate)

print("=" * 80)
print("JSON REPAIR COST ESTIMATE")
print("=" * 80)
print()
print(json.dumps(estimate_data, indent=2))
print()
print("=" * 80)

📝 Calling estimation tool directly to retrieve JSON estimate...

JSON REPAIR COST ESTIMATE

{
  "estimate_metadata": {
    "claim_number": "CL-2023-1156789",
    "estimate_date": "2025-10-22T16:29:28.816874",
    "disclaimer": "This is a preliminary estimate. Actual repair costs may vary based on hidden damage, parts availability, and labor rates."
  },
  "vehicle_information": {
    "year": 2020,
    "make": "Honda",
    "model": "Accord",
    "vin": "1HGCV2F35LA007149",
    "mileage": 35650,
    "vehicle_type": "Car"
  },
  "damage_assessment": {
    "damaged_components": [
      {
        "component": "Rear bumper",
        "damage_description": "Rear bumper dented and paint scratched, trunk lid misaligned, possible frame damage, brake lights damaged",
        "estimated_cost": 1150.0
      },
      {
        "component": "Trunk",
        "damage_description": "Rear bumper dented and paint scratched, trunk lid misaligned, possible frame damage, brake lights damaged",
        "estima

In [13]:
# Display a formatted summary of the estimate
print("=" * 80)
print("REPAIR COST ESTIMATE SUMMARY")
print("=" * 80)

# Vehicle Information
vehicle_info = estimate_data['vehicle_information']
print(f"\n Vehicle: {vehicle_info['year']} {vehicle_info['make']} {vehicle_info['model']}")
print(f"   Type: {vehicle_info['vehicle_type']}")
print(f"   VIN: {vehicle_info['vin']}")
print(f"   Mileage: {vehicle_info['mileage']:,} miles")

# Claim Information
metadata = estimate_data['estimate_metadata']
print(f"\n📋 Claim Number: {metadata['claim_number']}")
print(f"   Estimate Date: {metadata['estimate_date'][:10]}")

# Damaged Components
damage_assessment = estimate_data['damage_assessment']
print(f"\n Damaged Components:")
for component in damage_assessment['damaged_components']:
    print(f"   • {component['component']}: ${component['estimated_cost']:,.2f}")

print(f"\n   Subtotal: ${damage_assessment['subtotal']:,.2f}")

# Cost Adjustments
adjustments = estimate_data['cost_adjustments']
if adjustments['multipliers']:
    print(f"\n⚙️  Cost Adjustments:")
    for multiplier in adjustments['multipliers']:
        print(f"   • {multiplier['type'].replace('_', ' ').title()}: {multiplier['factor']}x")
        print(f"     Reason: {multiplier['reason']}")
    print(f"\n   Adjustment Total: ${adjustments['adjustment_total']:,.2f}")
else:
    print(f"\n⚙️  Cost Adjustments: None applied")

# Total Estimate
total = estimate_data['total_estimate']
print(f"\n TOTAL ESTIMATED REPAIR COST: ${total['amount']:,.2f} {total['currency']}")

print(f"\n  {metadata['disclaimer']}")
print("\n" + "=" * 80)

REPAIR COST ESTIMATE SUMMARY

 Vehicle: 2020 Honda Accord
   Type: Car
   VIN: 1HGCV2F35LA007149
   Mileage: 35,650 miles

📋 Claim Number: CL-2023-1156789
   Estimate Date: 2025-10-22

 Damaged Components:
   • Rear bumper: $1,150.00
   • Trunk: $1,850.00
   • Taillights: $450.00

   Subtotal: $3,450.00

⚙️  Cost Adjustments: None applied

 TOTAL ESTIMATED REPAIR COST: $3,450.00 USD

  This is a preliminary estimate. Actual repair costs may vary based on hidden damage, parts availability, and labor rates.

